# AEMO 2024 ISP Progressive Change Model - Execution

This notebook executes the AEMO 2024 Integrated System Plan (ISP) Progressive Change scenario using PyPSA.

## Model Overview

**AEMO 2024 ISP Progressive Change** represents Australia's National Electricity Market (NEM) including:
- Generation resources across all Australian states
- Transmission infrastructure (interconnectors)
- Variable renewable energy (VRE) profiles
- Multi-period investment optimization (2025-2050)

This model uses the **per-node strategy** where each demand file maps to a network bus (traditional approach).

## Configuration

- **Optimization Year**: 2025 (single year from multi-period model)
- **Output File**: `solved_network_2025.nc`
- **Model Type**: Electricity network with investment periods
- **Demand Strategy**: Per-node assignment

**Note**: While AEMO is a multi-period model (2025, 2030, 2035, 2040, 2050), we solve only for 2025 to minimize computation time.

---

## Setup and Imports

In [ ]:
import sys
import time
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent.parent))


from plexos_to_pypsa_converter.workflow import run_model_workflow

## Model Configuration

We'll configure the AEMO 2024 ISP Progressive Change model to solve for **year 2025 only**. The model will:

1. Load the PLEXOS XML data and time series
2. Convert to PyPSA format with per-node strategy
3. Optimize for the 2025 investment period
4. Save results to a NetCDF file

**Output Location**: `../results/aemo-2024-isp-progressive-change/solved_network_2025.nc`

In [ ]:
# Configuration
model_id = "aemo-2024-isp-progressive-change"
optimize_year = 2025
output_filename = "solved_network_2025.nc"
results_dir = Path.cwd().parent / "results" / model_id
output_path = results_dir / output_filename

print(f"Model ID: {model_id}")
print(f"Optimization Year: {optimize_year}")
print(f"Output file: {output_path}")
print(f"Results directory: {results_dir}")

## Execute Model Solve

This cell will execute the complete workflow:
- Load PLEXOS data
- Create PyPSA network
- Run optimization for year 2025
- ~~Save results~~ (skipped - we'll save manually with custom filename below)

**Note on Network Saving:**

By default, `run_model_workflow()` automatically saves the network to `src/examples/results/[model]/solved_network.nc`. 

To avoid duplicate files, we modify the workflow to **skip the automatic save step** and instead save once with our custom filename (`solved_network_2025.nc`). This gives us better control over the output filename while avoiding unnecessary duplicate saves.

**Note**: This may take several minutes depending on the model size and your system. The solve log will be displayed below.

**AEMO Context**: The AEMO model includes:
- Multiple NEM regions (QLD, NSW, VIC, SA, TAS)
- High renewable penetration scenarios
- Interstate interconnectors

In [ ]:
# Track solve time
start_time = time.time()

print("=" * 70)
print(f" Starting AEMO 2024 ISP Progressive Model Solve (Year {optimize_year})")
print("=" * 70)
print()

# Create modified workflow that skips the automatic save step
# We'll save manually with a custom filename below
from plexos_to_pypsa_converter.db.registry import MODEL_REGISTRY

default_workflow = MODEL_REGISTRY[model_id]["processing_workflow"]
modified_workflow = default_workflow.copy()

# Remove the save_network step (we'll save manually with custom filename)
modified_workflow["steps"] = [
    step for step in default_workflow["steps"] if step["name"] != "save_network"
]

print("Modified workflow: removed automatic save step (will save manually)")
print()

# Run the workflow with year parameter and without automatic save
# Note: optimize__year parameter specifies which investment period to optimize
# The workflow skips the automatic save to src/examples/results/[model]/solved_network.nc
network, setup_summary = run_model_workflow(
    model_id, workflow_overrides=modified_workflow, optimize__year=optimize_year
)

# Calculate solve time
solve_time = time.time() - start_time

print()
print("=" * 70)
print(" Model Solve Complete")
print("=" * 70)
print(f"Total time: {solve_time / 60:.2f} minutes")

## Save Results

Now we'll save the solved network to our custom filename. This is the **only save** - we skipped the automatic save step above to avoid creating duplicate files.

In [ ]:
# Save network to specified output file
results_dir.mkdir(parents=True, exist_ok=True)
network.export_to_netcdf(str(output_path))

print(f"✓ Network saved to: {output_path}")
print(f"  File size: {output_path.stat().st_size / 1024 / 1024:.2f} MB")

## Solve Summary and Verification

Let's verify the solve succeeded and display key statistics.

In [ ]:
# Display network statistics
print("=" * 70)
print(" Network Statistics")
print("=" * 70)
print()
print(f"Buses:           {len(network.buses)}")
print(f"Generators:      {len(network.generators)}")
print(f"Loads:           {len(network.loads)}")
print(f"Links:           {len(network.links)}")
print(f"Storage Units:   {len(network.storage_units)}")
print(f"Stores:          {len(network.stores)}")
print()
print(f"Snapshots:       {len(network.snapshots)}")

# Check if multi-period
if hasattr(network, "investment_periods") and len(network.investment_periods) > 0:
    print(f"Investment Periods: {len(network.investment_periods)}")
    print(f"  Optimized for: {optimize_year}")
else:
    print(f"Optimization Year: {optimize_year}")

print(f"Carriers:        {len(network.carriers)}")
print()

# Display objective value if available
try:
    objective = network.objective
    print(f"Objective Value: ${objective:,.0f}")
except:
    print("Objective Value: Not available")

print()
print(f"Solve Time:      {solve_time / 60:.2f} minutes")
print()
print("=" * 70)

## Carrier Overview

Display the unique carriers (energy types) in the AEMO model:

In [ ]:
# Show unique carriers
carriers = network.generators.carrier.unique().tolist()
print(f"Total unique carriers: {len(carriers)}")
print()
print("Carriers in AEMO model:")
for i, carrier in enumerate(sorted(carriers), 1):
    print(f"  {i:2d}. {carrier}")

## Regions

Display the buses representing Australian regions:

In [ ]:
# Show buses (NEM regions)
print(f"Total buses: {len(network.buses)}")
print()
print("Regions (Buses):")
for i, bus in enumerate(network.buses.index, 1):
    print(f"  {i}. {bus}")

---

## Next Steps

✓ Model solved successfully for year 2025!

✓ Results saved to: `solved_network_2025.nc`

**Continue to**: `aemo_analysis.ipynb` for detailed analysis and visualization of these results.

---